In [3]:
import pandas as pd
import requests
import time #needed to pause between API requests

# API Request

## Documentation

911 calls for service data:
https://data.detroitmi.gov/datasets/detroitmi::911-calls-for-service/explore<br>

API:<br>
ESRI Docs: https://developers.arcgis.com/documentation/ (general reference for ESRI)<br>
Developers- Socrata AND ESRI
https://data.detroitmi.gov/pages/developers<br>
API Explorer (query commands): https://developers.arcgis.com/rest/services-reference/enterprise/query-feature-service-layer-.htm<br>

Detroit example:
https://machine.domain.com/webadaptor/rest/services/Earthquakes/EarthquakesFromLastSevenDays/FeatureServer/0/query?where=magnitude+%3E+4.5&outFields=*&returnGeometry=true&returnIdsOnly=false&f=html

### Define API Request

In [4]:
url = "https://services2.arcgis.com/qvkbeam7Wirps6zC/arcgis/rest/services/911_Calls_New/FeatureServer/0/query"
parameters = {
    "where": "category in ('MNTLARM','MNTLNARM','MENTPPRS','WBC','SUICIP','SUICTHRT')",
    "outFields": "incident_id,agency,priority,category,call_timestamp,officerinitiated,respondingunit",
    "outSR":"4326",
    "f":"json",
    "resultRecordCount":1000,
    "resultOffset":0
}

### Retrieve One Batch (1000 records)

In [ ]:
#simple request, this worked
#url = "https://services2.arcgis.com/qvkbeam7Wirps6zC/arcgis/rest/services/911_Calls_New/FeatureServer/0/query?where=1%3D1&outFields=incident_id,agency,priority,category,call_timestamp,officerinitiated,respondingunit&outSR=4326&f=json"

In [47]:
# #run the query
# response = requests.get(url, params=parameters)
# data_1_batch = response.json()
# #print (data_1_batch)

In [48]:
#look at the data

#data_1_batch
#data_1_batch['fields']
#data_1_batch['features']

In [49]:
#use .pop command to open up each list of dictionaries
# list_calls = [{**call.pop('attributes')} for call in data_1_batch['features']]
# df_detroit_api = pd.DataFrame(list_calls)
# df_detroit_api

### Retrieve Multiple Batches (1000 records at a time)

In [43]:
#test with a small batch
# all_data = []

# i = 1
# while i <5:
#     response = requests.get(url, params=parameters)
#     data = response.json()
    
#     if not data.get("features"):
#         break
    
#     all_data.extend(data["features"])
    
#     parameters["resultOffset"] += parameters["resultRecordCount"]
    
#     time.sleep(1) # Wait for 1 second before the next request

#     i+=1
    
# print(all_data)

In [5]:
#because Detroit limits me to 1000 records per request, I need to loop through repeated requests until done
#this takes about 5-10 minutes to run, with a 2 second pause for each of about 100 requests

all_data = []

while True:
    response = requests.get(url, params=parameters)
    data = response.json()
    
    if not data.get("features"):
        break
    
    all_data.extend(data["features"])
    
    parameters["resultOffset"] += parameters["resultRecordCount"]
    
    time.sleep(5) # Wait for 2 seconds before the next request

# print(all_data)

In [6]:
list_calls = [{**call.pop('attributes')} for call in all_data]
df_detroit_api = pd.DataFrame(list_calls)
df_detroit_api

,incident_id,agency,priority,category,call_timestamp,officerinitiated,respondingunit
0,201626402101,DPD,3,MENTPPRS,1474413140000,No,
1,201626402108,DPD,3,WBC,1474413282000,No,A0703
2,201626401001,DPD,3,MENTPPRS,1474391088000,No,D1007
3,201626401066,DPD,3,MENTPPRS,1474392442000,No,
4,201626401162,DPD,3,MENTPPRS,1474394806000,No,A0304
...,...,...,...,...,...,...,...
103262,202323600572,DPD,2,MNTLNARM,1692861034000,No,M0302
103263,202323601099,DPD,1,MNTLARM,1692879011000,No,D1211
103264,202323601138,DPD,2,MNTLNARM,1692879686000,No,D0709
103265,202323601197,DPD,2,MNTLNARM,1692880851000,No,D0502


In [7]:
df_detroit_api.to_csv("2023_08_25_detroit_911_mh.csv")